Multi Persona AI Writing Assistant – Prototype

This notebook:

- Defines several AI personas for writing feedback
- Sends a student paragraph to each persona
- Combines their feedback into a single final response
- Saves all outputs to a table you can analyse later


In [13]:
!pip install openai pandas --quiet

import os
import pandas as pd
from datetime import datetime
from openai import OpenAI

# Set your API key in a safe way.
# For quick testing only, you can uncomment the next line and paste your key.
# Do not commit your key to GitHub.

os.environ["OPENAI_API_KEY"] = ""

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


Define personas

In [14]:
PERSONAS = {
    "critic": {
        "role": "system",
        "content": (
            "You are a strict writing critic. "
            "Point out weaknesses in structure, clarity and cohesion. "
            "Focus on specific problems. Do not give praise."
        ),
    },
    "explainer": {
        "role": "system",
        "content": (
            "You explain things step by step. "
            "Give concrete suggestions and simple examples the writer can follow."
        ),
    },
    "supporter": {
        "role": "system",
        "content": (
            "You are encouraging but honest. "
            "Highlight what works first, then suggest two or three clear improvements."
        ),
    },
    "planner": {
        "role": "system",
        "content": (
            "You help the writer plan a better structure. "
            "Suggest an outline and ordering of ideas."
        ),
    },
}


Single persona feedback function

In [15]:
def get_persona_feedback(persona_name: str, essay: str, rubric: str = "") -> str:
    persona = PERSONAS[persona_name]

    user_prompt = (
        "Here is a student paragraph.\n\n"
        f"{essay}\n\n"
        "Give feedback the student can act on directly."
    )
    if rubric:
        user_prompt += f"\n\nUse this rubric as a guide:\n{rubric}"

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            persona,
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.4,
    )
    return response.choices[0].message.content.strip()


Multi persona orchestration

In [16]:
def run_multi_persona_session(essay: str, rubric: str = "") -> dict:
    outputs = {}

    for name in PERSONAS.keys():
        fb = get_persona_feedback(name, essay, rubric)
        outputs[name] = fb

    combined_prompt = (
        "You are a writing coach.\n\n"
        "You will see feedback from several assistants with different roles "
        "(critic, explainer, supporter, planner).\n\n"
        "Feedback from each assistant is tagged below.\n\n"
    )

    for name, fb in outputs.items():
        combined_prompt += f"[{name.upper()} FEEDBACK]\n{fb}\n\n"

    combined_prompt += (
        "Based on all of this, write one clear set of feedback for the student. "
        "Make it structured and easy to follow, with short numbered points."
    )

    combined_response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You merge feedback from multiple assistants into a single response."},
            {"role": "user", "content": combined_prompt},
        ],
        temperature=0.4,
    )

    outputs["combined"] = combined_response.choices[0].message.content.strip()
    return outputs


tasks and rubric

In [17]:
example_essays = [
    "School uniforms should be banned because students need freedom to express themselves, "
    "but some people think uniforms help with discipline and focus. I believe they are unfair "
    "because not everyone feels comfortable wearing the same thing and it does not always "
    "improve behaviour.",

    "Climate change is getting worse and many countries still burn fossil fuels even though "
    "they know it is harmful. Governments talk about action but real changes are slow. "
    "People recycle and try to reduce waste, but it feels small compared to what big companies do.",
]

rubric_text = (
    "Focus on four aspects: clarity, structure, strength of argument, and suggestions for improvement. "
    "Feedback should be specific enough that the student can edit the paragraph after reading it."
)


Run sessions and collect results

In [18]:
rows = []

for i, essay in enumerate(example_essays, start=1):
    outputs = run_multi_persona_session(essay, rubric=rubric_text)
    for persona, fb in outputs.items():
        rows.append(
            {
                "essay_id": i,
                "persona": persona,
                "essay": essay,
                "feedback": fb,
                "rubric": rubric_text,
            }
        )

results_df = pd.DataFrame(rows)
results_df


,essay_id,persona,essay,feedback,rubric
0,1,critic,School uniforms should be banned because stude...,1. Clarity: The paragraph mixes two opposing v...,"Focus on four aspects: clarity, structure, str..."
1,1,explainer,School uniforms should be banned because stude...,Here is feedback you can use to improve your p...,"Focus on four aspects: clarity, structure, str..."
2,1,supporter,School uniforms should be banned because stude...,You’ve done a great job stating your opinion c...,"Focus on four aspects: clarity, structure, str..."
3,1,planner,School uniforms should be banned because stude...,Here is some feedback to help you improve your...,"Focus on four aspects: clarity, structure, str..."
4,1,combined,School uniforms should be banned because stude...,"Here is some clear, structured feedback to hel...","Focus on four aspects: clarity, structure, str..."
5,2,critic,Climate change is getting worse and many count...,1. Clarity: The paragraph contains vague phras...,"Focus on four aspects: clarity, structure, str..."
6,2,explainer,Climate change is getting worse and many count...,Here is detailed feedback based on your paragr...,"Focus on four aspects: clarity, structure, str..."
7,2,supporter,Climate change is getting worse and many count...,You’ve made a strong start by clearly stating ...,"Focus on four aspects: clarity, structure, str..."
8,2,planner,Climate change is getting worse and many count...,Here is some feedback you can use to improve y...,"Focus on four aspects: clarity, structure, str..."
9,2,combined,Climate change is getting worse and many count...,"Here is clear, combined feedback to help you i...","Focus on four aspects: clarity, structure, str..."


Save outputs to CSV

In [19]:
from datetime import datetime

output_file = "persona_feedback_outputs.csv"
results_df.to_csv(output_file, index=False)
output_file



'persona_feedback_outputs.csv'